This notebook attempts to recreate the treatment groups since seed was not set during original group assignment, using the email outcomes from GovDelivery, the email vendor we used for this analysis.

In [1]:
import sys
sys.path.append('..')
import os.path

import pandas as pd
import numpy as np

DATA_DIR = os.path.join('..', 'data')

## Basic  Email 

This section pulls in the email outcomes from GovDelivery for the employees in the basic email treatment group, and in addition to cleaning the files, this section assigns everyone in this group a 1 as their true treatment arm assignment.

In [2]:
to_merge = [
    'Basic_email_outcomes_012318',
    'Basic_email_outcomes_021218',
    'Basic_email_outcomes_030118'
]

## Importing Data
basic = pd.read_csv(os.path.join(DATA_DIR, 'Basic_email_outcomes_011818.csv'))

basic.columns = ['Destination Address', 'Delivery Status_0118', 'Failure Message_0118', 
                 'Opens0118', 'Clicks0118', 'Click0118_1', 'Click0118_2']

## Transforming email to upper case for consistency
basic['Destination Address'] = basic['Destination Address'].str.upper()

## Assigning treatment designation
basic['treatment_real'] = 1

## Delete folks with emails designated 'NETWORK SUBSCRIBER'
basic = basic[basic['Destination Address']!='NETWORK SUBSCRIBER']

#Drop delivery failures - they should not be included in our total
print (basic['Delivery Status_0118'].value_counts())
print ('\nBefore drop:', len(basic))
basic = basic[basic['Delivery Status_0118'] == 'Delivered']
print('After drop:', len(basic))
print()


## Reading in and cleaning in files via loops
# the whole: i[-6:-2] refers to the date format in the file names, 
# and giving those same dates to our columns so we can distinguish clicks from one another
# -6 = sixth char from end
# -2 = second char from end

for i in to_merge:
    add = pd.read_csv(os.path.join(DATA_DIR, 'Basic_email_outcomes_'+i[-6:-2]+'18.csv'))

    ncol = add.shape[1]
    nclick = ncol - 5
    cnames = ['Destination Address', 'Delivery Status_'+i[-6:-2], 
             'Failure Message_'+i[-6:-2], 'Opens'+i[-6:-2], 'Clicks'+i[-6:-2]]
    
    for n in range(1,nclick+1):
        cnames.append('Click'+i[-6:-2]+'_'+str(n))
            
    add.columns = cnames        
    add['Destination Address'] = add['Destination Address'].str.upper()
    add = add[add['Destination Address']!='NETWORK SUBSCRIBER']
    
    
    for n in range(1, nclick+1):
        add['Click'+i[-6:-2]+'_'+str(n)] = add['Click'+i[-6:-2]+'_'+str(n)].str[:-4]
    
    add['in_'+i[-6:-2]] = 1
    print(add['Delivery Status_'+i[-6:-2]].value_counts(), '\n')
    
    print()
    print('Dropping delivery failures:')
    
    print('Before drop:', len(add))
    add = add[add['Delivery Status_'+i[-6:-2]] == 'Delivered']
    print('After drop:', len(add))                  
        
    ## Merging 
    temp = basic.merge(add, how = 'left', on = ['Destination Address'])
    temp['Destination Address'].value_counts(ascending = False).head()
    print('Number of rows: '+str(temp.shape[0]))
        
    basic = temp
    
print('')    
print(basic.shape)

Delivered           11230
Delivery Failure        1
Name: Delivery Status_0118, dtype: int64

Before drop: 11231
After drop: 11230

Delivered           11230
Delivery Failure        1
Name: Delivery Status_0123, dtype: int64 


Dropping delivery failures:
Before drop: 11231
After drop: 11230
Number of rows: 11230
Delivered           11213
Delivery Failure        3
Name: Delivery Status_0212, dtype: int64 


Dropping delivery failures:
Before drop: 11216
After drop: 11213
Number of rows: 11230
Delivered           11213
Delivery Failure        3
Name: Delivery Status_0301, dtype: int64 


Dropping delivery failures:
Before drop: 11216
After drop: 11213
Number of rows: 11230

(11230, 36)


## Simplified Choice Email

This section pulls in the email outcomes from GovDelivery for the employees in the simplified choice email treatment group, and in addition to cleaning the files, this section assigns everyone in this group a 2 as their true treatment arm assignment.

In [3]:
to_merge = [
    'Enhance_email_outcomes_012318',
    'Enhance_email_outcomes_021218',
    'Enhance_email_outcomes_030118'
]

#initialize dataframes with first email group
enhance = pd.read_csv(os.path.join(DATA_DIR, 'Enhance_email_outcomes_011818.csv'))

enhance.columns = ['Destination Address', 'Delivery Status_0118', 'Failure Message_0118', 
                   'Opens0118', 'Clicks0118', 'Click0118_1', 'Click0118_2']

#Change casing
enhance['Destination Address'] = enhance['Destination Address'].str.upper()

#Real treatment group assignment
enhance['treatment_real'] = 2
enhance = enhance[enhance['Destination Address']!='NETWORK SUBSCRIBER']

print (enhance['Delivery Status_0118'].value_counts())

print('\nBefore drop:', len(enhance))
enhance = enhance[enhance['Delivery Status_0118']== 'Delivered']
print('After drop', len(enhance))
print()


## Reading in and cleaning in files via loops
# the whole: i[-6:-2] refers to the date format in the file names, 
# and giving those same dates to our columns so we can distinguish clicks from one another
# -6 = sixth char from end
# -2 = second char from end

for i in to_merge:
    print(i)
    
    add = pd.read_csv(os.path.join(DATA_DIR, i+'.csv'))

    ncol = add.shape[1]
    nclick = ncol - 5
    cnames = ['Destination Address', 'Delivery Status_'+i[-6:-2], 'Failure Message_'+i[-6:-2], 
              'Opens'+i[-6:-2], 'Clicks'+i[-6:-2]]
    
    for n in range(1,nclick+1):
        cnames.append('Click'+i[-6:-2]+'_'+str(n))
        
    add.columns = cnames        
    add['Destination Address'] = add['Destination Address'].str.upper()
    add = add[add['Destination Address']!='NETWORK SUBSCRIBER']
    
    for n in range(1, nclick+1):
        add['Click'+i[-6:-2]+'_'+str(n)] = add['Click'+i[-6:-2]+'_'+str(n)].str[:-4]
    
    #Add a 1 to represent that it was in this particular csv
    add['in_'+i[-6:-2]] = 1
    
    print(add['Delivery Status_'+i[-6:-2]].value_counts(),'\n')
    
    print()
    print('Dropping delivery failures:')
    print('Before drop:', len(add))
    add = add[add['Delivery Status_' + i[-6:-2]] == 'Delivered']
    print('After drop:', len(add))
    
    
    ## Merging 
    temp = enhance.merge(add, how = 'left', on = ['Destination Address'])
    temp['Destination Address'].value_counts(ascending = False).head()
    print('Number of rows: '+str(temp.shape[0]))
        
    enhance = temp
print('')    
#print(enhance.shape)
#print(enhance.email_upper.value_counts().head())

Delivered           11234
Delivery Failure        1
Name: Delivery Status_0118, dtype: int64

Before drop: 11235
After drop 11234

Enhance_email_outcomes_012318
Delivered           11234
Delivery Failure        1
Name: Delivery Status_0123, dtype: int64 


Dropping delivery failures:
Before drop: 11235
After drop: 11234
Number of rows: 11234
Enhance_email_outcomes_021218
Delivered           11214
Delivery Failure        2
Name: Delivery Status_0212, dtype: int64 


Dropping delivery failures:
Before drop: 11216
After drop: 11214
Number of rows: 11234
Enhance_email_outcomes_030118
Delivered           11214
Delivery Failure        2
Name: Delivery Status_0301, dtype: int64 


Dropping delivery failures:
Before drop: 11216
After drop: 11214
Number of rows: 11234



## Control

This section pulls in the email outcomes from GovDelivery for the employees in the control group. However these people are the ones whose emails did not work for the email sends, so they will be dropped from our dataset.

In [ ]:
to_merge = [
    'Control_deleted_subscribers_021218',
    'control_unsubscribe_031618'
]

#Read in files
control = pd.read_csv(os.path.join(DATA_DIR, 'Control_Deleted_012318.csv'))
control.columns = ['Destination Address', 'temp', 'Failure Message_0123']

control['Destination Address'] = control['Destination Address'].str.upper()
control['Delivery Status_0123'] = 'Deleted'
control = control.drop(['temp'], axis=1)
print (control.shape)
print (control['Delivery Status_0123'].value_counts())
control.head().T

for i in to_merge:
    add = pd.read_csv(os.path.join(DATA_DIR, i +'.csv'))

    cnames = ['Destination Address', 'temp', 'Failure Message_'+i[-6:-2]]
        
    add.columns = cnames        
    add['Destination Address'] = add['Destination Address'].str.upper()

    add = add[add['Destination Address']!='NETWORK SUBSCRIBER']
    add['in_'+i[-6:-2]] = 1
        
    # Fix here so that we're appending, and not merging
    # Each file has a different email address, so merging doesn't do anything. 
    temp = control.append(add)
    temp['Destination Address'].value_counts(ascending = False).head()
    print('Number of rows: '+str(temp.shape[0]))
        
    control = temp
    control['treatment_real'] = 0
    
print('')    
print(control.shape)
# print(control['Destination Address'].value_counts())

# Output cleared since it contained emails

## Unsuccessful emails

Pull in the unsuccessful emails. When we uploaded the email lists to gov delivery, these were the folks who were unable to receive emails. Since we know which group they were supposed to have been assigned to, we need to remember to take them out of the control group later. 

In [5]:
#These were all of the unsuceessful emails
#Read: When we uploaded the email lists to gov delivery, these were the folks who were unable to receive emails

basic_unsuccessful = pd.read_csv(os.path.join(DATA_DIR, 'basic_unsuccessful.csv'))
simplified_unsuccessful = pd.read_csv(os.path.join(DATA_DIR, 'enhance_unsuccessful.csv'))
control_unsuccessful = pd.read_csv(os.path.join(DATA_DIR, 'control_unsuccessful.csv'))

print('Number of unsuccessful emails:\n')
print('Control:', len(control_unsuccessful))
print('Basic:', len(basic_unsuccessful))
print('Simplified:', len(simplified_unsuccessful))

#Concatenate all files
failures = basic_unsuccessful.append(simplified_unsuccessful).append(control_unsuccessful)

#change to a consistent casing
failures['destination'] = failures.destination.str.upper()

print('Total unsuccessful:', len(failures))

Number of unsuccessful emails:

Control: 175
Basic: 187
Simplified: 176
Total unsuccessful: 538


## Handling Duplicates

There are ~27 employees who have duplicate entries in the dataset. This is due to these employees being associated with the emails OSSE.HR@DC.GOV or OCTOQA1@DC.GOV or OSSE.DOTHR@DC.GOV. It turns out that these employees received both the basic and simplified choice treatments. As a result, we have decided to drop them from the dataset because we would not be able to determine whether their choice to enroll or unenroll in 457b is a result of which treatment arm.

In addition, somehow Lab staff were still included in these email sends, so we need to make sure to remove them.

In [6]:
#Handling bad OSSE data

#OSSE data + The list of Lab staff still active during the time of the study


bad_emails_list = ['OCTOQA1@DC.GOV', 
              'OSSE.HR@DC.GOV', 
              'OSSE.DOTHR@DC.GOV']

#### NOTE: Our private repo include the actual emails for Lab and DCHR staff, but due to PII
#### we do not include the actual list items here
lab_emails = []
dchr_emails = []

bad_emails_init = bad_emails_list + lab_emails + dchr_emails

#uppercase for consistency
bad_emails = [x.upper() for x in bad_emails_init]

In [7]:
#Combine the results from the two treatment groups 
all_email_outcomes = basic.append(enhance)

#Renaming column name, bc it was previously "email_upper" beforehand, 
#and I don't want to change the code in the other notebooks
all_email_outcomes.rename(columns={'Destination Address': 'email_upper'}, inplace = True)

#Remove the bad emails from our data.
all_email_outcomes = all_email_outcomes[~all_email_outcomes.email_upper.isin(bad_emails)]

#What does our data look like? 
print(all_email_outcomes.treatment_real.value_counts(sort = False))
print(all_email_outcomes.shape)

1    11216
2    11219
Name: treatment_real, dtype: int64
(22435, 36)


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## Join to baseline data

In [8]:
#Read in data
df_ids = pd.read_csv(os.path.join(DATA_DIR, 'baseline.csv'), dtype = {"EmplID":str})

#change casing for easy merging, plus rename
df_ids['email_upper'] = df_ids['Email ID'].str.upper()

#Some of the bad OSSE emails were in our baseline, so let's get rid of them. 
df_ids = df_ids[~df_ids.email_upper.isin(bad_emails)]

print('Original Data: Breakout by treatment arm:')
print(df_ids.groupby('treatment')['Enrollment'].value_counts(dropna = False))

print()
print('Total by treatment arm:')
#Totals by group
print(df_ids.treatment.value_counts(sort = False))

Original Data: Breakout by treatment arm:
treatment  Enrollment
0          NaN           7150
           457BEN        4262
1          NaN           7153
           457BEN        4261
2          NaN           7151
           457BEN        4262
Name: Enrollment, dtype: int64

Total by treatment arm:
0    11412
1    11414
2    11413
Name: treatment, dtype: int64


In [9]:
#Merge email data to our baseline data
#Indicator = True, so that the people who were not in our email outcomes will be assigned as our control group.
outcomes = pd.merge(df_ids, 
                    all_email_outcomes, 
                    how = 'left', 
                    on = 'email_upper',
                    indicator = True)

outcomes.loc[outcomes._merge =='left_only', 'treatment_real'] = 0

print(outcomes.shape)

(34239, 62)


In [10]:
#Remove the employees that were in our failed email delivery list from above
outcomes = (outcomes[~((outcomes.email_upper.isin(failures.destination.str.upper()))|\
                    (outcomes.email_upper.isin(control['Destination Address'])))])

print(outcomes.treatment_real.value_counts(sort = False) )
print()
print(outcomes.groupby('treatment_real')['Enrollment'].value_counts(dropna = False))

0.0    11258
1.0    11217
2.0    11223
Name: treatment_real, dtype: int64

treatment_real  Enrollment
0.0             NaN           7032
                457BEN        4226
1.0             NaN           7022
                457BEN        4195
2.0             NaN           7027
                457BEN        4196
Name: Enrollment, dtype: int64


In [11]:
outcomes = (outcomes[~outcomes.email_upper.duplicated(keep=False)])
print(outcomes.shape)

print()
print(outcomes.treatment_real.value_counts(sort = False))
print()
print(outcomes.groupby('treatment_real')['Enrollment'].value_counts(dropna = False))

(33686, 62)

0.0    11256
1.0    11215
2.0    11215
Name: treatment_real, dtype: int64

treatment_real  Enrollment
0.0             NaN           7031
                457BEN        4225
1.0             NaN           7021
                457BEN        4194
2.0             NaN           7021
                457BEN        4194
Name: Enrollment, dtype: int64


In [12]:
#Dropping unnecessary columns. 
outcomes = outcomes.drop(labels = ['treatment', '_merge', 'Email ID', 
                                   'Last Name', 'First Name', 'Empl Record'], 
                         axis = 1)

outcomes.to_csv(os.path.join(DATA_DIR, 'outcomes_id_emails.csv'), 
                index = False, header = True)